# Algoritm inteligent + small data

### 1. Descriere EDA a datelor:
- Continut:<br>
Dataset-ul contine **4920** de exemple, unde fiecare input are **17** atribute (*Simptom_1, Simptom_2, ..., Simpotom_17*)<br>
Dataset-ul are ca scop corelarea unor simptome cu un **diagnostic** (variabila tinta). Atributele sunt de tip string, reprezentand un simptom asociat diagnosticului din linia respectiva

- Analiza statistica:<br>
Dataset-ul cuprinde 41 de diagnostice unice si 131 de simptome distincte. Dataset-ul este echilibrat, avand pentru fiecare diagnostic 120 de exemple.<br>
Analizand mai atent setul de date, am observat ca multe dintre perechile (colectie_simptome - diagnostic) sunt duplicate. Daca nu modific in niciun fel setul de date, atunci exista riscul ca procesul de testare al modelului dupa antrenare sa fie sabotat - exista sansa ca multe dintre exemplele din train set sa apara si in test set (**data leakage**), ceea ce ar duce la o valoare a acuratetii irelevanta (am testat pastrand duplicatele si am obtinut o acuratete de **100%**, evident nerealista)

In [4]:
import pandas as pd

df = pd.read_csv('data/DiseaseAndSymptoms.csv')

# Transformarea dataset-ului in perechi de tipul (lista_simptome - diagnostic)

symptom_cols = [col for col in df.columns if col.startswith("Symptom_")]
df["lista_simptome"] = df[symptom_cols].apply(lambda x: [s.strip() for s in x.dropna().astype(str) if s.strip() != ''], axis=1)

print(df[["Disease", "lista_simptome"]].head())
print()

# determinare simptome unice

all_symptoms = [s for lista in df['lista_simptome'] for s in lista if isinstance(lista, list)]
unique_symptoms = set(all_symptoms)

print(f'{len(unique_symptoms)} unique symptoms')

            Disease                                     lista_simptome
0  Fungal infection  [itching, skin_rash, nodal_skin_eruptions, dis...
1  Fungal infection  [skin_rash, nodal_skin_eruptions, dischromic _...
2  Fungal infection  [itching, nodal_skin_eruptions, dischromic _pa...
3  Fungal infection          [itching, skin_rash, dischromic _patches]
4  Fungal infection         [itching, skin_rash, nodal_skin_eruptions]

131 unique symptoms


#### Preprocesare date:
- voi elimina toate liniile duplicate (pentru a masura capacitatea de generalizare a modelului);
- pentru fiecare diagnostic, voi asocia un vector binar care sa ilustreze corelatia dintre un diagnostic, respectiv un simptom (1, daca simptomul este prezent, 0 altfel)
- voi codifica fiecare diagnostic cu un intreg

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

# eliminare duplicate

df['tuple_simptome'] = df['lista_simptome'].apply(lambda x: tuple(sorted(x)))
df_unique = df.drop_duplicates(subset=['Disease', 'tuple_simptome'])

print(df_unique.shape)
print(df_unique.head())

# one-hot encoding pentru simptome

mlb = MultiLabelBinarizer()
X = mlb.fit_transform(df_unique['tuple_simptome'])

KeyError: 'lista_simptome'